In [1]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

KEY_FILE_LOCATION = '/home/jzou/key_files/dbm_api_jubapluscc@gmail.com.json'
SCOPES = ['https://www.googleapis.com/auth/doubleclickbidmanager']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION, scopes=SCOPES)

# Build the service object.
service = build('doubleclickbidmanager', 'v1', credentials=credentials)

# create query, only need to run once

In [115]:
query_resourse = {
    'kind': 'doubleclickbidmanager#query',
    'metadata': {
        'dataRange': 'CUSTOM_DATES',
        'format': 'CSV',
        'title': 'JUBA_day'
    },
    'params': {
        'filters': [
            {'type': 'FILTER_ADVERTISER', 'value': '1958747'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13906955'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13857627'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13857390'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13857628'},
        ],
        'groupBys': [
            'FILTER_ADVERTISER',
            'FILTER_LINE_ITEM',
            'FILTER_ACTIVITY_ID',
            'FILTER_DAY_OF_WEEK'
        ],
        'includeInviteData': False,
        'metrics': [
            'METRIC_IMPRESSIONS',
            'METRIC_CLICKS',
            'METRIC_TOTAL_CONVERSIONS',
            'METRIC_REVENUE_ADVERTISER',
            'METRIC_MEDIA_COST_ADVERTISER',
            'METRIC_CTR'
        ],
        'type': 'TYPE_GENERAL'
    },
    'reportDataEndTimeMs': '1508025600000',
    'reportDataStartTimeMs': '1506297600000',
    'schedule': {
        'endTimeMs': '0',
        'frequency': 'ONE_TIME',
        'nextRunTimezoneCode': 'America/New_York'
    }
}

In [116]:
service.queries().createquery(body=query_resourse).execute()

{'kind': 'doubleclickbidmanager#query',
 'metadata': {'dataRange': 'CUSTOM_DATES',
  'format': 'CSV',
  'googleCloudStoragePathForLatestReport': '',
  'googleDrivePathForLatestReport': '',
  'latestReportRunTimeMs': '0',
  'reportCount': 0,
  'running': False,
  'sendNotification': False,
  'title': 'JUBA_day'},
 'params': {'filters': [{'type': 'FILTER_ADVERTISER', 'value': '1958747'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13906955'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13857627'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13857390'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13857628'},
   {'type': 'FILTER_PARTNER', 'value': '0'}],
  'groupBys': ['FILTER_ADVERTISER',
   'FILTER_LINE_ITEM',
   'FILTER_ACTIVITY_ID',
   'FILTER_DAY_OF_WEEK'],
  'includeInviteData': False,
  'metrics': ['METRIC_IMPRESSIONS',
   'METRIC_CLICKS',
   'METRIC_TOTAL_CONVERSIONS',
   'METRIC_REVENUE_ADVERTISER',
   'METRIC_MEDIA_COST_ADVERTISER',
   'METRIC_CTR'],
  'type': 'TYPE_GENERAL'},
 'queryId'

# list all queries created above

In [63]:
def list_queries():
    '''
    return:
        list of query_id
    '''
    queries = service.queries().listqueries().execute()
    return [x['queryId'] for x in queries['queries']]

In [117]:
query_ids = list_queries()

In [118]:
query_ids

['1510090985328',
 '1510094521954',
 '1510094797145',
 '1510154299818',
 '1510154794561',
 '1510174501175']

# run queries

In [120]:
def date_to_timestampMS(date_str):
    '''
    Args:
        date_str, eg. '2017-11-08'
    return:
        str, timestamp
    '''
    d = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return str(int((d-datetime.datetime(1970,1,1)).total_seconds()*1000))

def run_queries(query_ids, end_date):
    '''
    Args:
        query_ids, list of str
        end_date, str, eg. '2017-11-08'
    '''
    for query_id in query_ids:
        service.queries().runquery(queryId=query_id, body={
            'dataRange': 'CUSTOM_DATES',
            'reportDataEndTimeMs': '1508025600000',
            'reportDataStartTimeMs': date_to_timestampMS(end_date),
            "timezoneCode": 'America/New_York'
        }).execute()

In [121]:
run_queries(query_ids, '2017-11-08')

# download reports

In [112]:
from urllib.request import urlopen
from contextlib import closing

def download_reports(query_ids, output_dir):
    '''
    Args: 
        query_ids, list of str
        output_dir, str, eg. '/home/jzou'
    '''
    for query_id in query_ids:
        query = service.queries().getquery(queryId=query_id).execute()
        report_url = query['metadata']['googleCloudStoragePathForLatestReport']
        output_file = '%s/%s.csv' % (output_dir, query['metadata']['title'])
        with open(output_file, 'wb') as output:
            with closing(urlopen(report_url)) as url:
                output.write(url.read())

In [122]:
download_reports(query_ids, '/home/jzou')

In [114]:
query_ids

['1510090985328',
 '1510094521954',
 '1510094797145',
 '1510154299818',
 '1510154794561']